# Torsion check

In reinforced concrete design, torsion checks must be carried out together with shear interaction according to Eurocode 2. Torsion induces both shear stresses and twisting moments, which can lead to diagonal tension cracks in the concrete. Therefore, it is essential to ensure that the combined effect of shear and torsion is properly accounted for to maintain the structural integrity and safety of the element.

`Blueprints` provides a simple way to check the torsion for a reinforced cross-section using EN 1992-1-1:2004.
This example can be followed step by step by first importing the necessary classes and functions.

In [ ]:

from blueprints.checks.torsion_check.torsion_check import TorsionCheck
from blueprints.materials.concrete import ConcreteMaterial, ConcreteStrengthClass
from blueprints.materials.reinforcement_steel import ReinforcementSteelMaterial, ReinforcementSteelQuality
from blueprints.structural_sections.concrete.covers import CoversRectangular
from blueprints.structural_sections.concrete.reinforced_concrete_sections.rectangular import RectangularReinforcedCrossSection

from blueprints.unit_conversion import KN_TO_N, NMM_TO_KNM

Define the concrete material properties to be used in the calculation:

In [6]:
concrete = ConcreteMaterial(concrete_class=ConcreteStrengthClass.C35_45)

Define the steel material properties to be used in the calculation:

In [7]:
steel = ReinforcementSteelMaterial(steel_quality=ReinforcementSteelQuality.B500B)

Define a rectangular reinforced cross-section:

In [8]:
cs = RectangularReinforcedCrossSection(
    width=400,
    height=600,
    covers=CoversRectangular(upper=40, right=40, lower=40, left=40),
    concrete_material=concrete,
)

Add reinforcement to the cross-section:

In [9]:
cs.add_longitudinal_reinforcement_by_quantity(
    n=2,
    diameter=16,
    edge="upper",
    material=steel,
)
cs.add_longitudinal_reinforcement_by_quantity(
    n=2,
    diameter=20,
    edge="lower",
    material=steel,
)

cs.add_stirrup_along_edges(diameter=10, distance=200, material=steel, shear_check=True, torsion_check=False)
cs.add_stirrup_along_edges(diameter=10, distance=75, material=steel, shear_check=False, torsion_check=True)

Stirrup (id=2)|⌀10/B500B

Define forces and some initial data:

In [10]:
# Shear force in N
v_ed = 327 * KN_TO_N  

# Torsion moment in Nmm
t_ed = 60 / NMM_TO_KNM  

# Concrete compressive stress in cross-section due to axial loading and/or prestressing, assuming zero for this example
sigma_cp = 0

# The area of the tensile reinforcement, which extends lbd + d beyond the section considered
a_sl = 0

Run the check:

In [ ]:
torsion_check = TorsionCheck(cs=cs, sigma_cp=sigma_cp, a_sl=a_sl, v_ed=v_ed, t_ed=t_ed).check()

Results:
Minimum tensile reinforcement area: 0.57
Maximum longitudinal reinforcement area: 0.11
Minimum shear reinforcement ratio: 0.13
Maximum shear stirrup spacing: 0.49
Maximum torsion stirrup spacing: 0.30
Stirrup reinforcement for shear and torsion: 0.89
Concrete strut capacity: 0.59
Torsion moment capacity: 4.83
Torsion moment capacity is not enough. Additional reinforcement is required to resist this combination of shear and torsion forces.
Required additional longitudinal reinforcement for torsion (to be distributed along beam edges): 780.36 mm²
The required area of the additional longitudinal reinforcement due to shear: 376.05 mm²


True

You could also use the `TorsionCheck` to get a latex representation of the calculation by using:

In [ ]:
print(torsion_check.latex())